Combine all 5 metrics - drought risk, wildfire hazard risk, wind power, gas fuel, and oil fuel
- combine on counties - make a new county if there aren't any overlaps. na values for some of the metrics are okay.

Normalizing Scores:
- Using Min-Max method because we want scores to range from 0-1.
- Since the data is so heavily skewed, log transform first then do min-max

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [16]:
# WIND
raw_wind = pd.read_csv('Wind.csv')
wind = raw_wind[["Capacity (MW)", "County", "State", "Latitude", "Longitude"]].copy()
wind.head(5)

# OIL
raw_oil = pd.read_csv("Oil_Fuel.csv")
oil = raw_oil[["Subnational unit (province, state)", "Latitude", "Longitude", "Quantity (converted)"]].copy()
oil.head(5)

# GAS
raw_gas = pd.read_csv("Gas_Fuel.csv")
gas = raw_gas[["Subnational unit (province, state)", "Latitude", "Longitude", "Quantity (converted)"]].copy()
gas.head(5)

,"Subnational unit (province, state)",Latitude,Longitude,Quantity (converted)
0,Alaska,70.331718,-147.879900,3868.137817
1,Alaska,60.968647,-151.331333,37.408637
2,Alaska,NaN,NaN,27.036387
3,Alaska,60.772233,-151.503800,2.857681
4,Alaska,59.850499,-151.567667,3.095568


In [17]:
scaler = MinMaxScaler()

# WIND
wind['Capacity_log'] = np.log1p(wind['Capacity (MW)'])
wind['Capacity_normalized'] = scaler.fit_transform(wind[['Capacity_log']])
wind.head(5)

,Capacity (MW),County,State,Latitude,Longitude,Capacity_log,Capacity_normalized
0,250.0,Ellis County,Oklahoma,36.4870,-99.7390,5.525453,0.674386
1,174.8,Adair County,Iowa,41.4440,-94.6789,5.169347,0.624688
2,13.5,Mower County,Minnesota,43.5516,-92.7173,2.674149,0.276464
3,58.0,Adams County,Iowa,40.9239,-94.7163,4.077537,0.472318
4,50.7,Rush County,Kansas,38.4296,-99.5371,3.945458,0.453885


In [20]:
# GAS
gas['Quantity_log'] = np.log1p(gas['Quantity (converted)'])
gas['Quantity_normalized'] = scaler.fit_transform(gas[['Quantity_log']])
gas.head(5)

,"Subnational unit (province, state)",Latitude,Longitude,Quantity (converted),Quantity_log,Quantity_normalized
0,Alaska,70.331718,-147.879900,3868.137817,8.260787,0.764759
1,Alaska,60.968647,-151.331333,37.408637,3.648282,0.283633
2,Alaska,NaN,NaN,27.036387,3.333503,0.250799
3,Alaska,60.772233,-151.503800,2.857681,1.350066,0.043909
4,Alaska,59.850499,-151.567667,3.095568,1.409906,0.050151


In [21]:
# OIL
oil['Quantity_log'] = np.log1p(oil['Quantity (converted)'])
oil['Quantity_normalized'] = scaler.fit_transform(oil[['Quantity_log']])
oil.head(5)

,"Subnational unit (province, state)",Latitude,Longitude,Quantity (converted),Quantity_log,Quantity_normalized
0,Alaska,61.188517,-151.027400,0.000000,0.000000,0.000000
1,Alaska,70.324819,-149.945492,33.361528,3.536938,0.772550
2,Alaska,70.474762,-149.525500,12.162669,2.577385,0.562962
3,Alaska,70.309122,-148.670664,78.550002,4.376386,0.955906
4,Louisiana,32.458782,-93.555326,0.000526,0.000526,0.000115


In [33]:
# Combine the datasets by latitude and longitude. 
# Each observation should have a column for each metric. If a pair of lat long are the same, it works.

# gas and oil
fuel = pd.merge(
    gas, 
    oil, 
    on=['Latitude', 'Longitude'], 
    how='outer',
    suffixes=('_gas', '_oil')
)
fuel.head(5)

# remove unnecessary columns
fuel = fuel.rename(columns = {'Subnational unit (province, state)_gas' : 'State_Gas'})
fuel = fuel.rename(columns = {'Subnational unit (province, state)_oil' : 'State_Oil'})

fuel = fuel[["State_Gas", "State_Oil", "Latitude", "Longitude", "Quantity_normalized_gas", "Quantity_normalized_oil"]]

fuel = fuel.rename(columns = {'Quantity_normalized_gas' : 'Quantity_Gas'})
fuel = fuel.rename(columns = {'Quantity_normalized_oil' : 'Quantity_Oil'})
fuel.head(5)

,State_Gas,State_Oil,Latitude,Longitude,Quantity_Gas,Quantity_Oil
0,NaN,Federal offshore,26.102250,-92.061500,NaN,0.578501
1,Federal offshore,NaN,26.114667,-94.910333,0.589351,NaN
2,Federal offshore,Federal offshore,26.168000,-92.133000,0.328396,0.497244
3,Federal offshore,NaN,26.191667,-91.440667,0.314919,NaN
4,Federal offshore,Federal offshore,26.401000,-91.357500,0.284081,0.552446


In [40]:
# combine fuel and wind
combined = pd.merge(
    fuel, 
    wind, 
    on=['Latitude', 'Longitude'], 
    how='outer',
    #suffixes=('_gas', '_oil')
)
combined.head(5)

# clean up data
combined = combined[["State_Gas", "State_Oil", "Latitude", "Longitude", "Quantity_Gas", "Quantity_Oil", "Capacity_normalized", "County"]]
combined = combined.rename(columns = {'Capacity_normalized' : 'Wind Plant Capacity', 'Quantity_Gas':'Gas Production Quantity', 'Quantity_Oil':'Oil Production Quantity'})
combined.head(5)

,State_Gas,State_Oil,Latitude,Longitude,Gas Production Quantity,Oil Production Quantity,Wind Plant Capacity,County
0,NaN,NaN,18.97800,-155.6880,NaN,NaN,0.334645,Hawaii County
1,NaN,NaN,20.80010,-156.5390,NaN,NaN,0.334645,Maui County
2,NaN,NaN,21.66920,-157.9501,NaN,NaN,0.371260,Honolulu County
3,NaN,NaN,21.68040,-157.9820,NaN,NaN,0.382505,Honolulu County
4,NaN,Federal offshore,26.10225,-92.0615,NaN,0.578501,NaN,NaN


In [42]:
# reorder columns
combined = combined.iloc[:, [0, 1, 7, 2, 3, 4, 5, 6]]
combined.head(5)

,State_Gas,State_Oil,County,Latitude,Longitude,Gas Production Quantity,Oil Production Quantity,Wind Plant Capacity
0,NaN,NaN,Hawaii County,18.97800,-155.6880,NaN,NaN,0.334645
1,NaN,NaN,Maui County,20.80010,-156.5390,NaN,NaN,0.334645
2,NaN,NaN,Honolulu County,21.66920,-157.9501,NaN,NaN,0.371260
3,NaN,NaN,Honolulu County,21.68040,-157.9820,NaN,NaN,0.382505
4,NaN,Federal offshore,NaN,26.10225,-92.0615,NaN,0.578501,NaN


In [43]:
combined.to_csv("OilGasWind.csv", index=False)